In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
pip install surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 5.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp310-cp310-linux_x86_64.whl size=2357247 sha256=7ad9b1ad88252541f104f92238c4a03e971e34b34179bd2f17afd69548c012eb
  Stored in directory: /root/.cache/pip/wheels/4b/3f/df/6acbf0a40397d9bf3ff97f582cc22fb9ce66adde75bc71fd54
Successfully built scikit-surprise


In [3]:
import pandas as pd

# Cargar los datos
movies = pd.read_csv('/content/drive/MyDrive/MineriaDeDatos_Examen/movies_metadata.csv')
ratings = pd.read_csv('/content/drive/MyDrive/MineriaDeDatos_Examen/ratings.csv')

# Verificar los datos
print(movies.head())
print(ratings.head())


<ipython-input-3-3e55876bd905>:4: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movies = pd.read_csv('/content/drive/MyDrive/MineriaDeDatos_Examen/movies_metadata.csv')


   adult                              belongs_to_collection    budget  \
0  False  {'id': 10194, 'name': 'Toy Story Collection', ...  30000000   
1  False                                                NaN  65000000   
2  False  {'id': 119050, 'name': 'Grumpy Old Men Collect...         0   
3  False                                                NaN  16000000   
4  False  {'id': 96871, 'name': 'Father of the Bride Col...         0   

                                              genres  \
0  [{'id': 16, 'name': 'Animation'}, {'id': 35, '...   
1  [{'id': 12, 'name': 'Adventure'}, {'id': 14, '...   
2  [{'id': 10749, 'name': 'Romance'}, {'id': 35, ...   
3  [{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...   
4                     [{'id': 35, 'name': 'Comedy'}]   

                               homepage     id    imdb_id original_language  \
0  http://toystory.disney.com/toy-story    862  tt0114709                en   
1                                   NaN   8844  tt0113497         

In [4]:
from scipy.sparse import csr_matrix

# Crear una matriz de puntuaciones dispersa
ratings_matrix = csr_matrix((ratings['rating'], (ratings['userId'], ratings['movieId'])))


In [5]:
# Filtrar usuarios y películas que tengan una cantidad mínima de interacciones
min_ratings = 50
user_counts = ratings['userId'].value_counts()
movie_counts = ratings['movieId'].value_counts()

active_users = user_counts[user_counts >= min_ratings].index
active_movies = movie_counts[movie_counts >= min_ratings].index

filtered_ratings = ratings[(ratings['userId'].isin(active_users)) & (ratings['movieId'].isin(active_movies))]

# Crear la matriz de puntuaciones filtrada
user_movie_matrix = csr_matrix((filtered_ratings['rating'], (filtered_ratings['userId'], filtered_ratings['movieId'])))


In [6]:
from sklearn.model_selection import train_test_split

# Dividir los datos en conjunto de entrenamiento y prueba
train_data, test_data = train_test_split(filtered_ratings, test_size=0.2, random_state=42)

# Crear matrices dispersas para los datos de entrenamiento y prueba
train_matrix = csr_matrix((train_data['rating'], (train_data['userId'], train_data['movieId'])))
test_matrix = csr_matrix((test_data['rating'], (test_data['userId'], test_data['movieId'])))


In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split
from scipy.sparse import csr_matrix

# Asegúrate de que 'userId' y 'movieId' sean de tipo int32 para reducir el uso de memoria
filtered_ratings['userId'] = filtered_ratings['userId'].astype('int32')
filtered_ratings['movieId'] = filtered_ratings['movieId'].astype('int32')

# Limitar el número de usuarios y películas para reducir el tamaño del dataset
max_users = 2000  # Ajusta este valor según tu memoria disponible
max_movies = 2000  # Ajusta este valor según tu memoria disponible

# Obtener los usuarios y películas más frecuentes
top_users = filtered_ratings['userId'].value_counts().head(max_users).index
top_movies = filtered_ratings['movieId'].value_counts().head(max_movies).index

# Filtrar el dataset para mantener solo los usuarios y películas más frecuentes
filtered_ratings_reduced = filtered_ratings[
    (filtered_ratings['userId'].isin(top_users)) & (filtered_ratings['movieId'].isin(top_movies))
]

# Tomar una muestra aleatoria del 10% de los datos para reducir aún más el tamaño
sampled_ratings = filtered_ratings_reduced.sample(frac=0.1, random_state=42)

# Dividir los datos en conjunto de entrenamiento y prueba
train_data, test_data = train_test_split(sampled_ratings, test_size=0.2, random_state=42)

# Crear matrices dispersas para los datos de entrenamiento y prueba
train_matrix = csr_matrix((train_data['rating'], (train_data['userId'], train_data['movieId'])))
test_matrix = csr_matrix((test_data['rating'], (test_data['userId'], test_data['movieId'])))


<ipython-input-7-0444e5914b3b>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_ratings['userId'] = filtered_ratings['userId'].astype('int32')
<ipython-input-7-0444e5914b3b>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_ratings['movieId'] = filtered_ratings['movieId'].astype('int32')


In [10]:
import pandas as pd
from sklearn.model_selection import train_test_split
from scipy.sparse import csr_matrix
from surprise import Dataset, Reader, KNNBasic
from surprise.model_selection import train_test_split as surprise_train_test_split
from surprise import accuracy

# Asegúrate de que 'userId' y 'movieId' sean de tipo int32 para reducir el uso de memoria
filtered_ratings['userId'] = filtered_ratings['userId'].astype('int32')
filtered_ratings['movieId'] = filtered_ratings['movieId'].astype('int32')

# Limitar el número de usuarios y películas para reducir el tamaño del dataset
max_users = 2000  # Ajusta este valor según tu memoria disponible
max_movies = 2000  # Ajusta este valor según tu memoria disponible

# Obtener los usuarios y películas más frecuentes
top_users = filtered_ratings['userId'].value_counts().head(max_users).index
top_movies = filtered_ratings['movieId'].value_counts().head(max_movies).index

# Filtrar el dataset para mantener solo los usuarios y películas más frecuentes
filtered_ratings_reduced = filtered_ratings[
    (filtered_ratings['userId'].isin(top_users)) & (filtered_ratings['movieId'].isin(top_movies))
]

# Tomar una muestra aleatoria del 10% de los datos para reducir aún más el tamaño
sampled_ratings = filtered_ratings_reduced.sample(frac=0.1, random_state=42)

# Dividir los datos en conjunto de entrenamiento y prueba
train_data, test_data = train_test_split(sampled_ratings, test_size=0.2, random_state=42)

# Crear el dataset para surprise
reader = Reader(rating_scale=(sampled_ratings['rating'].min(), sampled_ratings['rating'].max()))
data = Dataset.load_from_df(sampled_ratings[['userId', 'movieId', 'rating']], reader)

# Dividir el conjunto de datos en entrenamiento y prueba usando surprise
trainset, testset = surprise_train_test_split(data, test_size=0.2, random_state=42)

# Crear y entrenar el modelo
model = KNNBasic(k=5, sim_options={'name': 'cosine', 'user_based': True})
model.fit(trainset)

# Hacer predicciones
predictions = model.test(testset)

# Evaluar el modelo
rmse = accuracy.rmse(predictions)
print(f"RMSE: {rmse}")

# Ejemplo de cómo obtener recomendaciones para un usuario específico
def get_top_n(predictions, n=10):
    # First map the predictions to each user.
    top_n = {}
    for uid, iid, true_r, est, _ in predictions:
        if not top_n.get(uid):
            top_n[uid] = []
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and get the N highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

top_n_recommendations = get_top_n(predictions, n=10)
for user, recommendations in top_n_recommendations.items():
    print(f"User {user}: {recommendations}")


<ipython-input-10-43cc257d051c>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_ratings['userId'] = filtered_ratings['userId'].astype('int32')
<ipython-input-10-43cc257d051c>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_ratings['movieId'] = filtered_ratings['movieId'].astype('int32')


Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 0.9946
RMSE: 0.9946126695412154
User 50958: [(904, 4.5), (44555, 4.4), (2599, 3.9), (1225, 3.8), (6708, 3.8), (2580, 3.5), (1219, 3.4), (81847, 3.2), (84772, 3.2), (56152, 3.2)]
User 251204: [(1209, 4.5), (4235, 4.5), (5618, 4.5), (1222, 4.2), (509, 4.2), (8464, 4.0), (4239, 4.0), (6942, 4.0), (778, 3.9), (3000, 3.9)]
User 100129: [(2944, 4.4), (593, 4.4), (3996, 4.0), (6867, 3.9), (61132, 3.9), (57669, 3.7), (97752, 3.6999343357529217), (1357, 3.6), (2908, 3.6), (145, 3.6)]
User 122726: [(1254, 4.5), (30749, 4.3), (930, 4.2), (1213, 4.1), (2186, 4.1), (1, 3.9), (2115, 3.8), (1961, 3.8), (2664, 3.7), (4306, 3.6)]
User 54872: [(1259, 4.3), (85414, 4.0), (7387, 3.9), (348, 3.8), (3448, 3.8), (25, 3.7), (3019, 3.7), (34162, 3.6), (3260, 3.5), (6297, 3.4)]
User 59482: [(527, 4.4), (104913, 3.899646329912116), (2918, 3.8), (1873, 3.7), (102125, 3.6), (30812, 3.6), (4467, 3.3), (1587, 3.1), (163, 3.0), (1608, 3